# 第7回 その1: 音声データの入出力
今回はデータ処理の応用として音声データを扱います。  
ここでは python を使った音声データの入出力について簡単に紹介します。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/07"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/07/
# 現在のフォルダの中身を表示
%ls

`input.wav`というデータが表示されていることを確認してください。

## ステップ1: 音声データの読み込み
音声ファイルには様々な形式（mp3やwavなど）がありますが，ここでは<font color='red'>**wavファイル**</font>を扱います。  
pythonで wavファイルの入出力を行うライブラリとして，<font color='red'>**wave**</font>があります。  
以下で，waveライブラリをインポートします。また，お馴染みのnumpyとmatplotlibもインポートします。

In [ ]:
import wave as wave # ちなみにこの行はas以降を省略して import wave だけでも良い。
import numpy as np
import matplotlib.pyplot as plt

waveライブラリを使って`input.wav`以下のコードによって開きます。

In [ ]:
wav_file = 'input.wav'
with wave.open(wav_file, 'rb') as wav:
  # サンプリング周波数 [Hz] を取得
  sampling_frequency = wav.getframerate()
  # その他の情報（ファイルサイズ等，書き込み時に必要）を取得
  wav_params = wav.getparams()
  # wavデータを読み込む
  waveform = wav.readframes(wav.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  waveform = np.frombuffer(waveform, dtype=np.int16).astype(np.float32)

上記の`sampling_frequency`は<font color='red'>**サンプリング周波数 [Hz]**</font>と呼び，１秒間に含まれるデータの数を表します。  
サンプリング周波数を表示させてみます。

In [ ]:
print('sampling frequency :' + str(sampling_frequency) + ' [Hz]')

8000 [Hz] と表示されました。  
つまり，この音声は１秒間に8000個の音声波形のデータが格納されているということです。  
言い換えると，1/8000秒間隔で音声波形を記録しているということです。

この音声波形をプロットします。

In [ ]:
# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(waveform)) # 横軸データ
plt.plot(x, waveform)
plt.show()

このwavファイルは16bit short型で記録されているので，縦軸の取り得る値の範囲は-32768～32767です（$2^{16} = 65536$）。    
横軸は１目盛につき1/8000秒で表現されています。  
この状態だと見にくいので，横軸を秒に換算して表示させます。  
これは上のソースコードの`x`をサンプリング周波数(8000)で割っておけば出来ます。  

In [ ]:
# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(waveform)) / sampling_frequency # 横軸データ
plt.plot(x, waveform)
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.show()

## ステップ2: 音声データの書き込み  
読み込んだ音声データを逆順に並び替えてみましょう。  
ベクトルデータの逆順への並び替えは，numpyの<font color='red'>flip関数</font>を使います。  

In [ ]:
# waveform を逆順に並び替える
waveform_rev = np.flip(waveform)

# データをプロット
plt.figure(figsize=(10,8))
plt.subplot(2, 1, 1)
plt.plot(x, waveform, label='waveform')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(x, waveform_rev, label='waveform_rev')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

作成した`waveform_rev`を`output.wav`というファイル名で書き込みます。  
書き込みは以下のソースコードのように行います。  

In [ ]:
out_file = 'output.wav'
with wave.open(out_file, 'wb') as out:
  # 音声データの情報（wav_params）をセット
  out.setparams(wav_params)
  # numpy array型(32bit float)のデータをバイナリデータ（16bit short）に変換
  out_binary_data = waveform_rev.astype(np.int16).tobytes()
  # データを書き込む
  out.writeframes(out_binary_data)

## ステップ3: Notebook上への音声ファイルの貼り付け  
以下のようにすることで，音声ファイルをNotebook上で聞くことも出来ます。

In [ ]:
import IPython.display
print('input.wav')
IPython.display.display(IPython.display.Audio('input.wav'))
print('output.wav')
IPython.display.display(IPython.display.Audio('output.wav'))

音声ファイルを沢山貼り付けすぎると，`convert_report.ipynb`でHTMLファイルに変換した際に音声ファイルが消える可能性があります。  
（HTMLファイルに変換できるipynbファイルの容量に限りがあるためです。）  
<font color='red'>レポート課題で音声ファイルを貼り付けた際は，convert_report.ipynbで作成されたHTMLファイルを必ず確認してから提出してください。</font>  
音声ファイルの貼り付け関連でトラブルがあった場合はご連絡ください。  
（htmlと分けて音声ファイルを提出するなどの指示をします。）